# Data Engineer Challenge 

In [ ]:
from pathlib import Path

from memory_profiler import profile

from q1_memory import q1_memory
from q1_time import q1_time


In [ ]:
%load_ext memory_profiler

In [ ]:
file_path = (
    Path().cwd().parent.absolute() / "data" / "farmers-protest-tweets-2021-2-4.json"
)

## Problema 1

### Optimización tiempo de ejecución

In [ ]:
%%memit
q1_time(file_path)


In [ ]:
%%timeit
q1_time(file_path)

### Optimización de memoria

In [ ]:
%%memit
q1_memory(file_path)

In [ ]:
%%timeit
q1_memory(file_path)

## Problema 2